<a href="https://colab.research.google.com/github/CarlosCordova2023/ML_M01_5/blob/main/ML_M01_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === PILA (LIFO) CON LISTA ===
stack = []                           # Creamos una lista vacía que usaremos como pila
stack.append("A")                    # push: apilamos 'A' al tope (final de la lista)
stack.append("B")                    # push: apilamos 'B' (operación O(1) amortizado)
tope = stack[-1]                     # peek: leemos el elemento del tope sin remover
salio = stack.pop()                  # pop: removemos y devolvemos el tope ('B') O(1)
print("stack:", stack, "| peek:", tope, "| pop:", salio)  # Estado y resultados


stack: ['A'] | peek: B | pop: B


In [ ]:
# === COLA (FIFO) CON DEQUE ===
from collections import deque        # deque es eficiente para operaciones en los extremos
q = deque()                          # Cola vacía
q.append("A")                        # enqueue: encolar 'A' (al final)
q.append("B")                        # enqueue: encolar 'B'
primero = q[0]                       # peek: mirar el primero sin sacarlo
salio = q.popleft()                  # dequeue: sacar 'A' (frente) O(1)
print("cola:", list(q), "| peek:", primero, "| dequeue:", salio)


In [2]:
# === HEAP MÍNIMO (PRIORIDAD) ===
import heapq                         # Módulo estándar para min-heap
heap = []                            # Lista que actuará como heap
heapq.heappush(heap, (3, "backup"))  # Insertar (prioridad=3, tarea="backup")
heapq.heappush(heap, (1, "deploy"))  # Menor número = más prioritario
heapq.heappush(heap, (2, "tests"))   # Insertar otro elemento
prio, tarea = heapq.heappop(heap)    # Extraer el mínimo (prioridad 1)
print("sale primero:", (prio, tarea)) # ('deploy')
print("heap restante:", heap)         # Vista interna (no orden total)


sale primero: (1, 'deploy')
heap restante: [(2, 'tests'), (3, 'backup')]


In [3]:
# === UNION–FIND (DSU) ===
class DSU:
    def __init__(self, n):
        self.parent = list(range(n))   # Cada nodo inicia como su propia raíz
        self.rank = [0]*n              # Rank ~ altura para unir eficientemente

    def find(self, x):
        if self.parent[x] != x:                       # Si x no es raíz…
            self.parent[x] = self.find(self.parent[x])# …comprimir camino (apuntar a raíz)
        return self.parent[x]                         # Retornar raíz

    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)          # Raíces de a y b
        if ra == rb: return False                    # Ya unidos → no hacer nada
        if self.rank[ra] < self.rank[rb]:            # Unir por rango (menor debajo de mayor)
            self.parent[ra] = rb
        elif self.rank[ra] > self.rank[rb]:
            self.parent[rb] = ra
        else:
            self.parent[rb] = ra; self.rank[ra] += 1 # Si empatan, cualquiera sube rango
        return True

# Ejemplo de uso:
dsu = DSU(5)
dsu.union(0,1); dsu.union(3,4)
print(dsu.find(0) == dsu.find(1))     # True  (conectados)
print(dsu.find(1) == dsu.find(2))     # False (no conectados)


True
False


In [4]:
# === TRIE (PREFIJOS) ===
class TrieNode:
    def __init__(self):
        self.hijos = {}                # char -> TrieNode
        self.es_fin = False            # Marca fin de palabra

class Trie:
    def __init__(self):
        self.raiz = TrieNode()         # Nodo raíz vacío

    def insertar(self, palabra):
        nodo = self.raiz               # Empezamos en raíz
        for ch in palabra:             # Avanzamos carácter por carácter
            if ch not in nodo.hijos:   # Si falta el hijo…
                nodo.hijos[ch] = TrieNode()  # …lo creamos
            nodo = nodo.hijos[ch]      # Descendemos
        nodo.es_fin = True             # Marcamos fin de palabra

    def _colectar(self, nodo, prefijo, out):
        if nodo.es_fin:                # Si aquí termina una palabra…
            out.append(prefijo)        # …la agregamos al resultado
        for ch, nxt in nodo.hijos.items():     # Exploramos hijos en profundidad
            self._colectar(nxt, prefijo + ch, out)

    def sugerencias(self, prefijo):
        nodo = self.raiz               # Volvemos a raíz
        for ch in prefijo:             # Descendemos siguiendo el prefijo
            if ch not in nodo.hijos:   # Si el prefijo no existe…
                return []              # …no hay sugerencias
            nodo = nodo.hijos[ch]
        res = []                       # Acumulador de resultados
        self._colectar(nodo, prefijo, res)  # Recolectamos palabras bajo el prefijo
        return res

# Ejemplo:
trie = Trie()
for w in ["data", "dataset", "date", "dog"]:
    trie.insertar(w)
print(trie.sugerencias("dat"))         # ['data', 'dataset', 'date']


['data', 'dataset', 'date']


In [5]:
# === BST: insertar/buscar O(h); h=altura (log n balanceado, n peor caso) ===
class Nodo:
    def __init__(self, val):
        self.val = val                 # Valor guardado en el nodo
        self.izq = None                # Hijo izquierdo
        self.der = None                # Hijo derecho

class BST:
    def __init__(self):
        self.raiz = None               # Árbol vacío

    def insertar(self, val):
        if not self.raiz:              # Si no hay raíz, creamos una
            self.raiz = Nodo(val); return
        cur = self.raiz                # Partimos en la raíz
        while True:                    # Buscamos posición de inserción
            if val < cur.val:          # Ir a izquierda
                if not cur.izq: cur.izq = Nodo(val); return
                cur = cur.izq
            else:                      # Ir a derecha (duplicados a derecha)
                if not cur.der: cur.der = Nodo(val); return
                cur = cur.der

    def buscar(self, val):
        cur = self.raiz                # Desde la raíz
        while cur:                     # Mientras haya nodos
            if val == cur.val: return True
            cur = cur.izq if val < cur.val else cur.der
        return False

    def inorder(self):
        out = []                       # Lista donde acumulamos
        def dfs(n):                    # Recorrido en-orden (izq, nodo, der)
            if not n: return
            dfs(n.izq); out.append(n.val); dfs(n.der)
        dfs(self.raiz); return out

# Ejemplo:
bst = BST()
for v in [7, 3, 9, 1, 5, 8, 10]:
    bst.insertar(v)
print(bst.inorder())                   # [1, 3, 5, 7, 8, 9, 10]
print(bst.buscar(5), bst.buscar(42))   # True, False


[1, 3, 5, 7, 8, 9, 10]
True False


In [6]:
# === BFS: distancias mínimas en número de aristas (no ponderado) O(V+E) ===
from collections import deque

G = {                                  # Grafo no dirigido como adyacencias
    "A": {"B", "C"},
    "B": {"A", "D"},
    "C": {"A", "D"},
    "D": {"B", "C", "E"},
    "E": {"D"},
}

def bfs_distancias(g, origen):
    dist = {origen: 0}                 # Distancia 0 al origen
    q = deque([origen])                # Cola inicial con el origen
    while q:                           # Mientras queden por visitar
        v = q.popleft()                # Sacamos el más antiguo (FIFO)
        for w in g.get(v, ()):         # Recorremos vecinos
            if w not in dist:          # Si no fue visitado
                dist[w] = dist[v] + 1  # Distancia = dist del padre + 1
                q.append(w)            # Encolamos para expandir luego
    return dist

print("BFS desde A:", bfs_distancias(G, "A"))

# === Dijkstra: costo mínimo en grafo ponderado (pesos >= 0) O((V+E) log V) ===
import heapq

W = {                                  # Grafo dirigido y ponderado
    "A": [("B", 2), ("C", 5)],
    "B": [("C", 1), ("D", 3)],
    "C": [("D", 1)],
    "D": [("E", 4)],
    "E": [],
}

def dijkstra(g, origen):
    dist = {origen: 0.0}               # Mejor costo al origen
    heap = [(0.0, origen)]             # Heap con pares (costo, nodo)
    visit = set()                      # Nodos ya fijados/optimizados
    while heap:                        # Mientras existan candidatos
        d, v = heapq.heappop(heap)     # Tomamos el de menor costo actual
        if v in visit:                 # Si ya lo fijamos, seguimos
            continue
        visit.add(v)                   # Marcamos como fijado
        for w, c in g.get(v, []):      # Recorremos aristas salientes
            nd = d + c                 # Costo alternativo por v->w
            if nd < dist.get(w, float("inf")):  # Si mejora el mejor conocido
                dist[w] = nd           # Actualizamos coste
                heapq.heappush(heap, (nd, w))   # Reencolamos para seguir
    return dist

print("Dijkstra desde A:", dijkstra(W, "A"))


BFS desde A: {'A': 0, 'B': 1, 'C': 1, 'D': 2, 'E': 3}
Dijkstra desde A: {'A': 0.0, 'B': 2.0, 'C': 3.0, 'D': 4.0, 'E': 8.0}


In [7]:
# === BENCHMARK RÁPIDO: DEQUE VS LIST COMO COLA ===
from collections import deque
import time

N = 20000                               # Tamaño de la prueba (ajusta según tu PC)

dq = deque()                            # Deque
t0 = time.perf_counter()
for i in range(N): dq.append(i)         # Encolar N veces (O(1) amort.)
for i in range(N): dq.popleft()         # Desencolar N veces (O(1) amort.)
t_deque = time.perf_counter() - t0

lst = []                                # List
t0 = time.perf_counter()
for i in range(N): lst.append(i)        # Encolar N veces (O(1) amort.)
for i in range(N): lst.pop(0)           # Desencolar desde el frente (O(n) cada vez)
t_list = time.perf_counter() - t0

print(f"Deque: {t_deque:.4f}s | List (cola): {t_list:.4f}s")


Deque: 0.0047s | List (cola): 0.0401s
